In [1]:
import os
import re
import pickle
import shutil
import zipfile
import collections
import numpy as np
from os import path
from datetime import datetime
from subprocess import check_output

In [2]:
def compress(source, target, name='result', max_size=np.inf, ignore=None):
    '''
        Input:
            source: 압축 대상 파일들이 모여있는 폴더 경로
            target: zip 파일을 저장할 경로
            max_size: 파일 1개당 최대 용량(byte 단위, 용량 초과 시 분할 압축)
            name: zip 파일 이름
            ignore: 무시할 파일명 리스트
            
        Output:
            zip 파일 (Return None)
        
        Description:
            - source에 있는 파일 전체를 압축한 후 target_path에 {name}.zip 파일로 저장함
            - 상대경로, 절대경로 둘 다 가능
            - 끝에 \ 붙여도 안 붙여도 상관없음
            - 경로가 존재하지 않으면 작동 안 함
            - source에 파일이 아닌 폴더가 있으면, 그 폴더 내부에 있는 파일을 모두 탐색하여 복사함
            - name 뒤에 .zip이 있어도 없어도 상관없음
            - 파일당 사이즈가 max_size보다 크면 error
            - 기존 zip 파일 위에 자동으로 덮어씀
            - target 폴더에 다른 zip 파일이 있지 않도록 하는 것을 추천
            
        Example:
            compress('C:/Users/Administrator/dev/source', 'C:/Users/Administrator/dev/result', 'test', 20000, ['test.txt', 'test2.txt'])
    '''
    if not name.endswith('.zip'):
        name = name + '.zip'
    source_path_abs_list = files_in_folder(path.abspath(source))
    if ignore != None:
        source_path_abs_list = [p for p in source_path_abs_list if path.basename(p) not in ignore]
    duplicates = [item for item, count in collections.Counter([path.basename(p) for p in source_path_abs_list]).items() if count > 1]
    if duplicates != []:
        raise Exception('중복되는 파일이 있습니다: {}'.format(duplicates))
    target_path_abs = path.abspath(target)
    result_zip = zipfile.ZipFile(target_path_abs + '\\' + name, 'w')
    file_size = 0
    i = 0
    for fp in source_path_abs_list:
        if(file_size + path.getsize(fp) > max_size):
            if path.getsize(fp) > max_size:
                raise Exception('파일당 사이즈가 max_size보다 큽니다: {} bytes'.format(path.getsize(fp)))
            result_zip.close()
            result_zip = zipfile.ZipFile(target_path_abs + '\\' + name[:-4] + '_{:03d}'.format(i+1) + '.zip', 'w')
            file_size = 0
            i += 1
        result_zip.write(fp, path.basename(fp), compress_type=zipfile.ZIP_DEFLATED)
        file_size += path.getsize(fp)
    result_zip.close()

def files_in_folder(p, full_path=True):
    '''
        Description:
            특정 폴더 안에 있는 모든 파일들의 절대경로 추출
        
        Input:
            source: 탐색하고자 하는 폴더의 경로
            full_path: 전체 경로 / 파일명만
        
        Output:
            폴더경로 list
        
        Example:
            files_in_folder('C:\\Users\\Administrator\\dev\\result')
    '''
    files = os.listdir(p)
    result = []
    for f in files:
        if not path.isdir(path.join(p, f)):
            if full_path:
                result.append(path.abspath(path.join(p, f)))
            else:
                result.append(f)
        else:
            result += files_in_folder(path.join(p, f), full_path)
    return result

def check_file_number(source):
    '''
        Description:
            files_in_folder가 파일을 잘 가져왔는지 체크
            
        Input:
            source: 체크할 폴더 경로
    '''
    n = len(files_in_folder(source))
    tree = check_output('tree /f {}'.format(source), shell=True).decode('ansi')
    m = sum([re.search('│?[^└├].+\.\w+', z) != None for z in tree.split('\r\n')])
    print('파일 수: {}개 (Path: {}, Checked: {})'.format(n, path.abspath(source), n==m))
    if n != m:
        raise Exception('파일 개수 검사를 통과하지 못했습니다.')

def check_duplicates(source, ignore=None):
    '''
        Description:
            source 경로에서 이름이 같은 파일명 있는지 체크
            
        Input:
            source: 검사하고자 하는 폴더 경로
            ignore: 무시하고자 하는 파일
        
        Example:
            check_duplicates('.', ['Thumbs.db'])
    '''
    files_list = files_in_folder(source, False)
    if ignore != None:
        files_list = [p for p in files_list if path.basename(p) not in ignore]
    duplicates = [item for item, count in collections.Counter(files_list).items() if count > 1]
    if duplicates != []:
        raise Exception('[Path: {}] 이름이 중복되는 파일이 있습니다: {}'.format(path.abspath(source), find_path(source, duplicates)))
    else:
        print('[Path: {}] 중복되는 파일이 없습니다.'.format(path.abspath(source)))

def find_path(source, files):
    '''
        Description:
            source 경로에 있는 files의 경로를 찾음
            
        Input:
            source: 찾고자 하는 폴더
            files: 파일들 목록
            
        Return:
            파일들의 절대경로 list
        
        Example:
            find_path('.', ['raas.ipynb'])
    '''
    result = [p for p in files_in_folder(source) if path.basename(p) in files]
    if result == []:
        print('[Path: {}] {} 파일이 없습니다.'.format(path.abspath(source), files))
    else:
        return result
    
    
def make_router(source, ignore=None, save=True):
    '''
        Description:
            - 폴더 내 각 파일들의 상대위치를 추출함
            - 라우터를 저장하게 되면 router.txt로 저장됨 (pickle)
        
        Input:
            source: 탐색하고자 하는 폴더의 경로
            ignore: 무시하고자 하는 파일명 리스트
            save: 만들어진 라우터 저장 여부
        
        Output:
            (파일명, 파일경로) dictionary
        
        Example:
            make_router('.', ['test.txt', 'test2.txt'])
    '''
    source_path_abs_list = files_in_folder(path.abspath(source))
    if ignore != None:
        source_path_abs_list = [p for p in source_path_abs_list if path.basename(p) not in ignore]
    duplicates = [item for item, count in collections.Counter([path.basename(p) for p in source_path_abs_list]).items() if count > 1]
    if duplicates != []:
        raise Exception('중복되는 파일이 있습니다: {}'.format(duplicates))
    router = {}
    for p in source_path_abs_list:
        router[path.basename(p)] = path.relpath(p, source)
    if save:
        now = datetime.now().strftime('%Y%m%d%H%M%S')
        with open('router_{}.txt'.format(now), 'wb') as f:
            pickle.dump(router, f)
    return router
    
def make_router_encrypt(source, save=True):
    '''
        Description:
            source 경로에 있는 files들의 이름을 변경한 후 router 생성
        Output:
            (변경된 파일명, 파일경로) dictionary
    '''
    
    if any([re.match('file_\d{4}', path.basename(p)) != None for p in files_in_folder(source)]):
        raise Exception('이미 한 번 실행되었습니다.')
    router = {}
    files = files_in_folder(source, True)
    for i, f in enumerate(files):
        renamed = 'file_{:04d}'.format(i) + path.splitext(f)[1]
        router[renamed] = path.relpath(f, source)
        os.rename(f, path.join(path.dirname(f), renamed))
    if save:
        now = datetime.now().strftime('%Y%m%d%H%M%S')
        with open('router_encrypt_{}.txt'.format(now), 'wb') as f:
            pickle.dump(router, f)
    return router

def mkdir2(target):
    '''
        Description:
            - 폴더 생성(이전 폴더 경로가 없으면 그 이전 폴더도 생성)
            - 파일 경로 넣어도 폴더 경로만 만듬
        
        Input:
            target: 폴더 경로
            
        Example:
            mkdir2('C:\\Users\\Administrator\\dev\\2\3\4') 
    '''
    try:
        os.mkdir(path.dirname(path.abspath(target)))
    except FileNotFoundError:
        mkdir2(path.join(path.abspath(target), '..'))
        os.mkdir(path.dirname(path.abspath(target)))
    except FileExistsError:
        pass
    
# def decrypt(source, router):
#     for file in files_in_folder(source, True):
#         os.rename(file, path.join(os.dirname(file), os.basename(router[os.basename(file)])))

def make_tree(source, target, router):
    '''
        Description:
            router를 따라 폴더 구조를 생성하고 파일을 그 안에 복사
        
        Input:
            source: 파일들이 들어있는 폴더 경로
            target: 트리를 만들 폴더 경로
            router: router (dictionary)
            
        Example:
            make_tree('Rockstar Games', 'tree', router)
    '''
    for p in router.values():
        mkdir2(path.join(target, p))
    for file in files_in_folder(source):
        try:
            filename = path.basename(file)
            shutil.copy(file, path.join(target, router[filename]))
        except KeyError:
            raise Exception('\'{}\' 의 경로명이 없습니다'.format(file))
            
def copy_to(source, target, ignore=None):
    '''
        Description:
            - source에 있는 모든 파일들을 target 폴더로 복사
            - 꼭 target 폴더 만들고 실행할 것
        
        Example:
            copy_to('CCleaner', 'zipfiles')
    '''
    source_path_abs_list = files_in_folder(path.abspath(source))
    if ignore != None:
        source_path_abs_list = [p for p in source_path_abs_list if path.basename(p) not in ignore]
    duplicates = [item for item, count in collections.Counter([path.basename(p) for p in source_path_abs_list]).items() if count > 1]
    if duplicates != []:
        raise Exception('중복되는 파일이 있습니다: {}'.format(duplicates))
    target_path_abs = path.abspath(target)
    if not path.exists(target_path_abs):
        raise Exception('경로 [{}] 가 존재하지 않습니다.'.format(target_path_abs))
    for fp in source_path_abs_list:
        shutil.copy(fp, target)
        
def decrypt(source, router):
    '''
        Description:
            source 경로에 있던 암호화 해제
            
        Example:
            router = make_router_encrypt('CCleaner')
            decrypt('CCleaner', router)
    '''
    for key, value in router.items():
        os.rename(path.join(path.join(source, path.dirname(value)), key), path.join(path.join(source, path.dirname(value)), path.basename(value)))

In [38]:
# 1. check_file_number
# 2. copy_to (warehouse)
# 3. make_router
# 4. make_router_encrypt
# 5. 보안을 다 해제해
# 6. compress (zipfiles)
# 7. 밖으로 빼(.zip, router, router_encrypt)
# 8. 압축 해제해
# 9. decrypt
# 10. make_tree

### 1. Example 1

In [5]:
source = 'C:\\CCleaner'

In [27]:
check_file_number(source)

파일 수: 5개 (Path: C:\Movegames, Checked: True)


In [29]:
copy_to(source, 'warehouse')

In [13]:
source = 'encrypted'
check_file_number(source)
make_router_encrypt(source)

파일 수: 15개 (Path: C:\Users\Administrator\dev\temp, Checked: True)


{'file_0000.dll': '비즈\\branding.dll',
 'file_0001.dat': '비즈\\CCleaner.dat',
 'file_0002.exe': '비즈\\CCleaner.exe',
 'file_0003.exe': '비즈\\CCleaner64.exe',
 'file_0004.dll': '비즈\\Lang\\lang-1042.dll',
 'file_0005.dll': '테크\\branding.dll',
 'file_0006.dat': '테크\\CCleaner.dat',
 'file_0007.exe': '테크\\CCleaner.exe',
 'file_0008.exe': '테크\\CCleaner64.exe',
 'file_0009.dll': '테크\\Lang\\lang-1042.dll',
 'file_0010.dll': '프로\\branding.dll',
 'file_0011.dat': '프로\\CCleaner.dat',
 'file_0012.exe': '프로\\CCleaner.exe',
 'file_0013.exe': '프로\\CCleaner64.exe',
 'file_0014.dll': '프로\\Lang\\lang-1042.dll'}

In [3]:
with open('router_encrypt_20190923013717.txt', 'rb') as f:
    router = pickle.load(f)

In [16]:
copy_to('encrypted', 'warehouse')

In [21]:
compress('warehouse', 'zipfiles', max_size=5e7)

In [4]:
router

{'file_0000.dll': '비즈\\branding.dll',
 'file_0001.dat': '비즈\\CCleaner.dat',
 'file_0002.exe': '비즈\\CCleaner.exe',
 'file_0003.exe': '비즈\\CCleaner64.exe',
 'file_0004.dll': '비즈\\Lang\\lang-1042.dll',
 'file_0005.dll': '테크\\branding.dll',
 'file_0006.dat': '테크\\CCleaner.dat',
 'file_0007.exe': '테크\\CCleaner.exe',
 'file_0008.exe': '테크\\CCleaner64.exe',
 'file_0009.dll': '테크\\Lang\\lang-1042.dll',
 'file_0010.dll': '프로\\branding.dll',
 'file_0011.dat': '프로\\CCleaner.dat',
 'file_0012.exe': '프로\\CCleaner.exe',
 'file_0013.exe': '프로\\CCleaner64.exe',
 'file_0014.dll': '프로\\Lang\\lang-1042.dll'}

In [6]:
make_tree('zipfiles', 'result', router)

### 2. Example 2

In [7]:
source = 'C:\\Movegames'

In [8]:
check_file_number(source)

파일 수: 5개 (Path: C:\Movegames, Checked: True)


In [14]:
copy_to(source, 'warehouse')

In [33]:
router = make_router(source)

{'D-Player.exe': 'Launcher\\D-Player.exe',
 'DownEngineSDK.dll': 'Launcher\\DownEngineSDK.dll',
 'Uninstaller.exe': 'Launcher\\Uninstaller.exe',
 'UpdateManager.exe': 'Launcher\\UpdateManager.exe',
 'vcredist_x86.exe': 'Launcher\\vcredist_x86.exe'}

In [36]:
with open('router_20190923014846.txt', 'rb') as f:
    router = pickle.load(f)

In [38]:
make_router_encrypt('warehouse')

{'file_0000.exe': 'D-Player.exe',
 'file_0001.dll': 'DownEngineSDK.dll',
 'file_0002.exe': 'Uninstaller.exe',
 'file_0003.exe': 'UpdateManager.exe',
 'file_0004.exe': 'vcredist_x86.exe'}

In [39]:
# 보안 해제

In [43]:
compress('warehouse', 'zipfiles', max_size=4e6)

In [ ]:
# 밖으로 뺌 (with router, router_encrypt)

In [3]:
with open('router_20190923014846.txt', 'rb') as f:
    router = pickle.load(f)
with open('router_encrypt_20190923015002.txt', 'rb') as f:
    router_encrypt = pickle.load(f)

In [4]:
router_encrypt

{'file_0000.exe': 'D-Player.exe',
 'file_0001.dll': 'DownEngineSDK.dll',
 'file_0002.exe': 'Uninstaller.exe',
 'file_0003.exe': 'UpdateManager.exe',
 'file_0004.exe': 'vcredist_x86.exe'}

In [ ]:
decrypt('zipfiles', router_encrypt)

In [9]:
make_tree('zipfiles', 'result', router)